In [1]:
import os

import cvxpy as cp
import dill
import numpy as np
import pandas as pd

%matplotlib inline

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

%load_ext autoreload

In [2]:
# load checkpoint 2 model
time = '400'
date = '2025-05-15'
experiment = 'NEW_NewGenes_checkpoint2'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [3]:
# get commonly stored variables
metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites = metabolism.metabolite_names.copy()
binary_kinetic_idx = metabolism.binary_kinetic_idx[0]
exchange_molecules = metabolism.exchange_molecules

S = stoichiometry.copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names )
homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).mean(axis=0)
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).mean(axis=0).copy()

In [4]:
# parameters that are the same across the two simulation
kinetic_reaction_ids = metabolism.kinetic_constraint_reactions
allowed_exchange_uptake = metabolism.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]
ADDED_RXNS = ['HS-TRANSPORT-RXN-CPD0-1202', 'HS-TRANSPORT-RXN-CPD0-1202 (reverse)',
                   'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE (reverse)',
                   'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE (reverse)',
                   'HS-BETA-GLUCURONID-RXN_CPD-3611//METOH', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN (reverse)',
                   'HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN','HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN (reverse)']

# Re-construct WC model's FBA problem

In [5]:
model = NetworkFlowModel(
            stoich_arr=S,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
model.set_up_exchanges(exchanges=exchange_molecules, uptakes=metabolism.allowed_exchange_uptake)

In [6]:
objective_weights={'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.000001}

binary_kinetic_idx = binary_kinetic_idx
force_flow_idx = None

v = cp.Variable(model.n_orig_rxns)
e = cp.Variable(model.n_exch_rxns)
dm = model.S_orig @ v + model.S_exch @ e
exch = model.S_exch @ e

constr = []
constr.append(dm[model.intermediates_idx] == 0)

if model.maintenance_idx is not None:
    constr.append(v[model.maintenance_idx] == maintenance)
# If enzymes not present, constrain rxn flux to 0
if binary_kinetic_idx is not None:
    constr.append(v[binary_kinetic_idx] == 0)
# If want to force flow through reactions, constrain rxn flux to 1 by idx
if force_flow_idx is not None:
    constr.append(v[force_flow_idx] >= 100)

homeostatic_concs = homeostatic_count * metabolism.counts_to_molar.asNumber()
homeostatic_dm_targets=np.array(list(dict(homeostatic).values()))
kinetic_targets=np.array(list(dict(kinetic).values()))
upper_bound = 1000000000

constr.extend([v >= 0, v <= upper_bound, e >= 0, e <= upper_bound])

loss = 0
# Must normalize by metabolite concentrations to prevent negative counts
loss += cp.norm1(
    (dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs
)
# flux_sum_part_obj = objective_weights["secretion"] * (cp.sum(e[self.secretion_idx]))

loss += (
    objective_weights["secretion"] * (cp.sum(e[model.secretion_idx]))
    if "secretion" in objective_weights
    else loss
)
loss += (
    objective_weights["efficiency"] * (cp.sum(v))
    if "efficiency" in objective_weights
    else loss
)

loss = (
    loss
    + objective_weights["kinetics"]
    * cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)
    if "kinetics" in objective_weights
    else loss
)

loss += (
    0.001 * cp.sum(cp.pos(100-v))
)

p = cp.Problem(cp.Minimize(loss), constr)
p.solve(solver=cp.GLOP, verbose=False)
print("status:", p.status)

status: optimal


objective: 108025.22625139347
number of nonzero velocities: 376
percentage of nonzero velocities: 4.011950490823731


In [65]:
print(objective_weights["efficiency"] * (cp.max(v)))

# result of changing efficiency term
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

1e-06 @ max(var118848, None, False)
objective: 108000.81988991922
number of nonzero fba reaction velocities: 511
number of nonzero base reaction velocities: 427
percentage of nonzero fba reaction velocities: 5.452411438326931
percentage of nonzero base reaction velocities: 15.260900643316655


In [66]:
np.max(velocities)

2257472.449663468

In [77]:
# result of changing efficiency and secretion term
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 102451.26223001747
number of nonzero fba reaction velocities: 563
number of nonzero base reaction velocities: 478
percentage of nonzero fba reaction velocities: 6.007255655142979
percentage of nonzero base reaction velocities: 17.08363116511794


In [87]:
# MOSEK MIP
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 102384.6717286675
number of nonzero fba reaction velocities: 6795
number of nonzero base reaction velocities: 1435
percentage of nonzero fba reaction velocities: 72.5032010243278
percentage of nonzero base reaction velocities: 51.28663330950679


In [12]:
# GLOP move away from sparse solution
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 108382.39703557317
number of nonzero fba reaction velocities: 6079
number of nonzero base reaction velocities: 1017
percentage of nonzero fba reaction velocities: 64.86342296201451
percentage of nonzero base reaction velocities: 36.34739099356683


In [74]:
# Original problem
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 108025.22625097341
number of nonzero fba reaction velocities: 375
number of nonzero base reaction velocities: 366
percentage of nonzero fba reaction velocities: 4.001280409731113
percentage of nonzero base reaction velocities: 13.080771979985704


In [75]:
np.max(velocities)

3987462.6782451863

# Define functions

In [8]:
def velocity_to_base_reaction(v):
    v = pd.DataFrame(v, index=reaction_names, columns=['flux'])
    v['base_reaction'] = v.index.map(fba_reaction_ids_to_base_reaction_ids)
    base_reaction_flux = v.groupby('base_reaction').sum()
    return base_reaction_flux

In [51]:
len(velocity_to_base_reaction(velocities))

2798

In [38]:
len(np.unique(np.array(fba_reaction_ids_to_base_reaction_ids.values())))

1

In [45]:
len(fba_reaction_ids_to_base_reaction_ids.values())

9391

In [53]:
velocity_to_base_reaction(velocities)

,flux
base_reaction,
1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN,0.00
1.1.1.127-RXN,0.00
1.1.1.215-RXN,0.00
1.1.1.251-RXN,0.00
1.1.1.271-RXN,0.00
...,...
XYLISOM-RXN,0.00
XYLONATE-DEHYDRATASE-RXN,0.00
XYLULOKIN-RXN,0.00
